<a href="https://colab.research.google.com/github/naenumtou/creditrisk/blob/main/factorCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mounth to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Change directory to Google Drive
%cd /content/drive/My Drive/Colab Notebooks/BScore

/content/drive/My Drive/Colab Notebooks/BScore


In [3]:
# Import libraries
import pandas as pd
import numpy as np

In [6]:
# Import data
transaction = pd.read_parquet('Usedcar_transaction.parquet', engine = 'pyarrow')
print(f'Total rows: {transaction.shape[0]}')
print(f'Total columns: {transaction.shape[1]}')

Total rows: 792456
Total columns: 31


In [7]:
# Sort transaction
transaction = transaction.sort_values(by = ['Br_Con', 'Monthkey'])

# Rename columns for factors creation
cols = ['OUTSTANDING_AMOUNT',
        'TOTAL_PAYMENT_MADE',
        'INSTALLMENT_AMOUNT',
        'PARTIAL_PAYMENT_FLAG',
        'DUE_AMOUNT',
        'OVERDUE_AMOUNT',
        'BUCKETS']

outCols = ['bal',
           'pay',
           'install',
           'pay_types',
           'due',
           'ovd',
           'del']

for col, outCol in zip(cols, outCols):
  transaction = transaction.rename(columns = {f'{col}': f'{outCol}'})
  print(f'Changed from {col} --> {outCol}')

Changed from OUTSTANDING_AMOUNT --> bal
Changed from TOTAL_PAYMENT_MADE --> pay
Changed from INSTALLMENT_AMOUNT --> install
Changed from PARTIAL_PAYMENT_FLAG --> pay_types
Changed from DUE_AMOUNT --> due
Changed from OVERDUE_AMOUNT --> ovd
Changed from BUCKETS --> del


In [8]:
# Factor creation
# Define month range
monthRanges = [12, 9, 6, 3]

# Lag backward factors
backwardFactors = ['del', 'due', 'ovd']

# Define lag range
lags = 12

for backwardFactor in backwardFactors:
  for i in range(1, lags + 1):
    transaction[f'{backwardFactor}{i}'] = transaction.groupby(['Br_Con'])[backwardFactor].shift(i)
    print(f'Factor {transaction.columns[-1]} created')

Factor del1 created
Factor del2 created
Factor del3 created
Factor del4 created
Factor del5 created
Factor del6 created
Factor del7 created
Factor del8 created
Factor del9 created
Factor del10 created
Factor del11 created
Factor del12 created
Factor due1 created
Factor due2 created
Factor due3 created
Factor due4 created
Factor due5 created
Factor due6 created
Factor due7 created
Factor due8 created
Factor due9 created
Factor due10 created
Factor due11 created
Factor due12 created
Factor ovd1 created
Factor ovd2 created
Factor ovd3 created
Factor ovd4 created
Factor ovd5 created
Factor ovd6 created
Factor ovd7 created
Factor ovd8 created
Factor ovd9 created
Factor ovd10 created
Factor ovd11 created
Factor ovd12 created


In [9]:
# Balance related facotors
# Average balance over the last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_bal_{i}'] = transaction.groupby(['Br_Con'])['bal'].rolling(window = i).mean().values
  print(f'Factor {transaction.columns[-1]} created')

# Average of balance to financed amount in the past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_bal_to_fin_{i}'] = transaction[f'avg_bal_{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')
    
# Average balance over the last 3 months to average balance over the last 12, 9, 6 months
for i in monthRanges:
  if i == 3:
    break
  else:
    condition = transaction[f'avg_bal_{i}'].isnull()
    transaction[f'avg_bal_3_to_avg_bal_{i}'] = np.where(condition, np.nan, transaction['avg_bal_3'] / transaction[f'avg_bal_{i}'])
    print(f'Factor {transaction.columns[-1]} created')
    
# Current balance to average balance over the last 12, 9, 6, 3 months
for i in monthRanges:
  condition = transaction[f'avg_bal_{i}'].isnull()
  transaction[f'bal_to_avg_bal_{i}'] = np.where(condition, np.nan, transaction['bal'] / transaction[f'avg_bal_{i}'])
  print(f'Factor {transaction.columns[-1]} created')
    
# Maximum balance in past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_bal_{i}'] = transaction.groupby(['Br_Con'])['bal'].rolling(window = i).max().values
  print(f'Factor {transaction.columns[-1]} created')

# Current balance to maximum balance in last 12, 9, 6, 3 months
for i in monthRanges:
  condition = transaction[f'max_bal_{i}'].isnull()
  transaction[f'bal_to_max_bal_{i}'] = np.where(condition, np.nan, transaction['bal'] / transaction[f'max_bal_{i}'])
  print(f'Factor {transaction.columns[-1]} created')

# Average balance over the last 12, 9, 6, 3 months to maximum balance in last 12, 9, 6, 3 months
for i in monthRanges:
  for j in monthRanges:
    if i < j:
      continue
    else:
      condition = transaction[f'max_bal_{i}'].isnull()
      transaction[f'avg_bal_{j}_to_max_bal_{i}'] = np.where(condition, np.nan, transaction[f'avg_bal_{j}'] / transaction[f'max_bal_{i}'])
      print(f'Factor {transaction.columns[-1]} created')
            
# Current balance to financed amount
transaction['bal_to_fin'] = transaction['bal'] / transaction['FINANCED_AMT']
print(f'Factor {transaction.columns[-1]} created')

# Minimum balance in past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'min_bal_{i}'] = transaction.groupby(['Br_Con'])['bal'].rolling(window = i).min().values
  print(f'Factor {transaction.columns[-1]} created')

Factor avg_bal_12 created
Factor avg_bal_9 created
Factor avg_bal_6 created
Factor avg_bal_3 created
Factor avg_bal_to_fin_12 created
Factor avg_bal_to_fin_9 created
Factor avg_bal_to_fin_6 created
Factor avg_bal_to_fin_3 created
Factor avg_bal_3_to_avg_bal_12 created
Factor avg_bal_3_to_avg_bal_9 created
Factor avg_bal_3_to_avg_bal_6 created
Factor bal_to_avg_bal_12 created
Factor bal_to_avg_bal_9 created
Factor bal_to_avg_bal_6 created
Factor bal_to_avg_bal_3 created
Factor max_bal_12 created
Factor max_bal_9 created
Factor max_bal_6 created
Factor max_bal_3 created
Factor bal_to_max_bal_12 created
Factor bal_to_max_bal_9 created
Factor bal_to_max_bal_6 created
Factor bal_to_max_bal_3 created
Factor avg_bal_12_to_max_bal_12 created
Factor avg_bal_9_to_max_bal_12 created
Factor avg_bal_6_to_max_bal_12 created
Factor avg_bal_3_to_max_bal_12 created
Factor avg_bal_9_to_max_bal_9 created
Factor avg_bal_6_to_max_bal_9 created
Factor avg_bal_3_to_max_bal_9 created
Factor avg_bal_6_to_max_b

In [10]:
# Payment related facotors
# Average repayment amount in past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_pay_{i}'] = transaction.groupby(['Br_Con'])['pay'].rolling(window = i).mean().values
  print(f'Factor {transaction.columns[-1]} created')
    
# Average of repayment amount to instalment amount in the past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_pay_to_install_{i}'] = transaction[f'avg_pay_{i}'] / transaction['install']
  print(f'Factor {transaction.columns[-1]} created')
    
# Maximum of repayment amount to installment in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_pay_to_install_{i}'] = transaction.groupby(['Br_Con'])['pay'].rolling(window = i).max().values
  transaction[f'max_pay_to_install_{i}'] = transaction[f'max_pay_to_install_{i}'] / transaction['install']
  print(f'Factor {transaction.columns[-1]} created')

# Minimum of repayment amount to installment in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'min_pay_to_install_{i}'] = transaction.groupby(['Br_Con'])['pay'].rolling(window = i).min().values
  transaction[f'min_pay_to_install_{i}'] = transaction[f'min_pay_to_install_{i}'] / transaction['install']
  print(f'Factor {transaction.columns[-1]} created')

# Number of times that payment was made in last 12, 9, 6, 3 months
# 1 = partial payment
for i in monthRanges:
  transaction['pay_check'] = transaction['pay_types'].eq(1).astype(int)
  transaction[f'n_partial_pay_{i}'] = transaction.groupby(['Br_Con'])['pay_check'].rolling(window = i).sum().values
  transaction = transaction.drop(['pay_check'], axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

# 2 = fully payment
for i in monthRanges:
  transaction['pay_check'] = transaction['pay_types'].eq(2).astype(int)
  transaction[f'n_fully_pay_{i}'] = transaction.groupby(['Br_Con'])['pay_check'].rolling(window = i).sum().values
  transaction = transaction.drop(['pay_check'], axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

# 3 = over payment
for i in monthRanges:
  transaction['pay_check'] = transaction['pay_types'].eq(3).astype(int)
  transaction[f'n_over_pay_{i}'] = transaction.groupby(['Br_Con'])['pay_check'].rolling(window = i).sum().values
  transaction = transaction.drop(['pay_check'], axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

# Number of full payments to number of partial payments in last 12, 9, 6, 3 months
for i in monthRanges:
  condition = ((transaction[f'n_fully_pay_{i}'] == 0) & (transaction[f'n_partial_pay_{i}'] == 0)) | (transaction[f'n_partial_pay_{i}'] == 0)              
  transaction[f'n_fully_to_n_partial_{i}'] = np.where(condition, -1, transaction[f'n_fully_pay_{i}'] / transaction[f'n_partial_pay_{i}'])
  print(f'Factor {transaction.columns[-1]} created')

# Create function to find longest consecutive
def maxConsecutive(window):
  diffs = np.diff(window, prepend = 0, append = 0)

  starts, = np.where(diffs == -1)
  ends, = np.where(diffs == 1)

  if len(ends) and len(starts):
    return (starts - ends).max()
  return 0

# Longest run of consecutive months which contract was fully repayment in the last 12, 9, 6, 3 months
for i in monthRanges:
  def consecutiveCount(data, w = i):
    return data.eq(2).rolling(w).apply(maxConsecutive)
  transaction[f'full_pmt_run_{i}'] = transaction.groupby('Br_Con')['pay_types'].transform(consecutiveCount)
  print(f'Factor {transaction.columns[-1]} created')
    
# Longest run of consecutive months which contract was any repayment (partial or fully) in the last 12, 9, 6, 3 months
for i in monthRanges:
  def consecutiveCount(data, w = i):
    return data.le(2).rolling(w).apply(maxConsecutive)
  transaction[f'any_pmt_run_{i}'] = transaction.groupby('Br_Con')['pay_types'].transform(consecutiveCount)
  print(f'Factor {transaction.columns[-1]} created')
    
# Repayment to due amount
transaction['pay_to_due'] = transaction['pay'] / transaction['due']

# Average of repayment amount to due amount in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_pay_to_due_{i}'] = transaction.groupby(['Br_Con'])['pay_to_due'].rolling(window = i).mean().values
  print(f'Factor {transaction.columns[-1]} created')

# Maximum of repayment amount to due amount in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_pay_to_due_{i}'] = transaction.groupby(['Br_Con'])['pay_to_due'].rolling(window = i).max().values
  print(f'Factor {transaction.columns[-1]} created')
    
# Minimum of repayment amount to due amount in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'min_pay_to_due_{i}'] = transaction.groupby(['Br_Con'])['pay_to_due'].rolling(window = i).min().values
  print(f'Factor {transaction.columns[-1]} created')

# Drop repayment to due amount
transaction = transaction.drop(['pay_to_due'], axis = 1)

Factor avg_pay_12 created
Factor avg_pay_9 created
Factor avg_pay_6 created
Factor avg_pay_3 created
Factor avg_pay_to_install_12 created
Factor avg_pay_to_install_9 created
Factor avg_pay_to_install_6 created
Factor avg_pay_to_install_3 created
Factor max_pay_to_install_12 created
Factor max_pay_to_install_9 created
Factor max_pay_to_install_6 created
Factor max_pay_to_install_3 created
Factor min_pay_to_install_12 created
Factor min_pay_to_install_9 created
Factor min_pay_to_install_6 created
Factor min_pay_to_install_3 created
Factor n_partial_pay_12 created
Factor n_partial_pay_9 created
Factor n_partial_pay_6 created
Factor n_partial_pay_3 created
Factor n_fully_pay_12 created
Factor n_fully_pay_9 created
Factor n_fully_pay_6 created
Factor n_fully_pay_3 created
Factor n_over_pay_12 created
Factor n_over_pay_9 created
Factor n_over_pay_6 created
Factor n_over_pay_3 created
Factor n_fully_to_n_partial_12 created
Factor n_fully_to_n_partial_9 created
Factor n_fully_to_n_partial_6 cr

In [11]:
# Due amount related factors
# Average due amount in the past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_due_{i}'] = transaction.groupby(['Br_Con'])['due'].rolling(window = i).mean().values
  print(f'Factor {transaction.columns[-1]} created')
    
# Maximum due amount in the past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_due_{i}'] = transaction.groupby(['Br_Con'])['due'].rolling(window = i).max().values
  print(f'Factor {transaction.columns[-1]} created')

# Current month due amount to average due amount over the last 12, 9, 6, 3 months
for i in monthRanges:
  condition = transaction[f'avg_due_{i}'].isnull()
  transaction[f'due_to_avg_due_{i}'] = np.where(condition, np.nan, transaction['due'] / transaction[f'avg_due_{i}'])
  print(f'Factor {transaction.columns[-1]} created')

# Average due amount over the last 3 months to average due amount over the last 12, 9, 6 months
for i in monthRanges:
  if i == 3:
    break
  else:
    condition = transaction[f'avg_due_{i}'].isnull()
    transaction[f'avg_due_3_to_avg_due_{i}'] = np.where(condition, np.nan, transaction['avg_due_3'] / transaction[f'avg_due_{i}'])
    print(f'Factor {transaction.columns[-1]} created')

# Create function to month after
def rank(window):
  newWindow = np.array(window)
  lastMax = np.where(newWindow == np.amax(newWindow))[0][-1]
  maxLocation = np.repeat(lastMax, newWindow.shape[0])
  location = np.arange(0, newWindow.shape[0])    
  ranks = location - maxLocation + 1
  return ranks[-1]

# Number of months since maximum due amount has been made in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'n_last_max_due_{i}'] = transaction.groupby('Br_Con')['due'].rolling(window = i).apply(rank).droplevel(0)
  print(f'Factor {transaction.columns[-1]} created')
    
# Due amount in current month to financed amount
transaction['due_to_fin'] = transaction['due'] / transaction['FINANCED_AMT']
print(f'Factor {transaction.columns[-1]} created')

# Due amount in previous 1-6 months to financed amount
for i in range(6, 0, -1):
  transaction[f'due_to_fin_{i}'] = transaction[f'due{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')

# Average of due amount to financed amount in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_due_to_fin_{i}'] = transaction[f'avg_due_{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')
    
# Maximum of due amount to financed amount in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_due_to_fin_{i}'] = transaction[f'max_due_{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')
    
# Overdue amount in current month to financed amount
transaction['ovd_to_fin'] = transaction['ovd'] / transaction['FINANCED_AMT']
print(f'Factor {transaction.columns[-1]} created')

# Overdue amount in previous 1-6 months to financed amount
for i in range(6, 0, -1):
  transaction[f'ovd_to_fin_{i}'] = transaction[f'ovd{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')

#Average overdue amount in past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_ovd_{i}'] = transaction.groupby(['Br_Con'])['ovd'].rolling(window = i).mean().values
  print(f'Factor {transaction.columns[-1]} created')
    
# Current month overdue amount to average overdue amount over the last 12, 9, 6, 3 months
for i in monthRanges:
  condition = transaction[f'avg_ovd_{i}'].isnull()
  transaction[f'ovd_to_avg_ovd_{i}'] = np.where(condition, np.nan, transaction['ovd'] / transaction[f'avg_ovd_{i}'])
  print(f'Factor {transaction.columns[-1]} created')
    
# Average overdue amount over the last 3 months to average overdue amount over the last 12, 9, 6 months
for i in monthRanges:
  if i == 3:
    break
  else:
    condition = transaction[f'avg_ovd_{i}'].isnull()
    transaction[f'avg_ovd_3_to_avg_ovd_{i}'] = np.where(condition, np.nan, transaction['avg_ovd_3'] / transaction[f'avg_ovd_{i}'])
    print(f'Factor {transaction.columns[-1]} created')

# Average of overdue amount to financed amount in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'avg_ovd_to_fin_{i}'] = transaction[f'avg_ovd_{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')
    
# Maximum overdue amount in past 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_ovd_{i}'] = transaction.groupby(['Br_Con'])['ovd'].rolling(window = i).max().values
  print(f'Factor {transaction.columns[-1]} created')

# Number of months since maximum overdue amount in last 12, 9, 6, 3 
for i in monthRanges:
  transaction[f'n_last_max_ovd_{i}'] = transaction.groupby('Br_Con')['ovd'].rolling(window = i).apply(rank).droplevel(0)
  print(f'Factor {transaction.columns[-1]} created')

# Maximum overdue amount in the past 12, 9, 6, 3 months to financed amount
for i in monthRanges:
  transaction[f'max_ovd_to_fin_{i}'] = transaction[f'max_ovd_{i}'] / transaction['FINANCED_AMT']
  print(f'Factor {transaction.columns[-1]} created')

Factor avg_due_12 created
Factor avg_due_9 created
Factor avg_due_6 created
Factor avg_due_3 created
Factor max_due_12 created
Factor max_due_9 created
Factor max_due_6 created
Factor max_due_3 created
Factor due_to_avg_due_12 created
Factor due_to_avg_due_9 created
Factor due_to_avg_due_6 created
Factor due_to_avg_due_3 created
Factor avg_due_3_to_avg_due_12 created
Factor avg_due_3_to_avg_due_9 created
Factor avg_due_3_to_avg_due_6 created
Factor n_last_max_due_12 created
Factor n_last_max_due_9 created
Factor n_last_max_due_6 created
Factor n_last_max_due_3 created
Factor due_to_fin created
Factor due_to_fin_6 created
Factor due_to_fin_5 created
Factor due_to_fin_4 created
Factor due_to_fin_3 created
Factor due_to_fin_2 created
Factor due_to_fin_1 created
Factor avg_due_to_fin_12 created
Factor avg_due_to_fin_9 created
Factor avg_due_to_fin_6 created
Factor avg_due_to_fin_3 created
Factor max_due_to_fin_12 created
Factor max_due_to_fin_9 created
Factor max_due_to_fin_6 created
Facto

In [12]:
# Delinquency related factors
# Maximum delinquency in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction[f'max_del_{i}'] = transaction.groupby(['Br_Con'])['del'].rolling(window = i).max().values
  print(f'Factor {transaction.columns[-1]} created')

# A contract has delinquency more than 0 day (del >= 1) in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction['delq_check'] = transaction['del'].ge(1).astype(int)
  transaction[f'ever_x_dpd_{i}'] = transaction.groupby(['Br_Con'])['delq_check'].rolling(window = i).max().values
  transaction = transaction.drop(['delq_check'], axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

# A contract has delinquency more than 30 days (del >= 2) in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction['delq_check'] = transaction['del'].ge(2).astype(int)
  transaction[f'ever_30_dpd_{i}'] = transaction.groupby(['Br_Con'])['delq_check'].rolling(window = i).max().values
  transaction = transaction.drop(['delq_check'], axis = 1)
  print(f'Factor {transaction.columns[-1]} created')
    
# A contract has delinquency more than 60 days (del >= 3) in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction['delq_check'] = transaction['del'].ge(3).astype(int)
  transaction[f'ever_60_dpd_{i}'] = transaction.groupby(['Br_Con'])['delq_check'].rolling(window = i).max().values
  transaction = transaction.drop(['delq_check'], axis = 1)
  print(f'Factor {transaction.columns[-1]} created')
    
# A contract has delinquency more than 90 days (del >= 4) in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction['delq_check'] = transaction['del'].ge(4).astype(int)
  transaction[f'ever_90_dpd_{i}'] = transaction.groupby(['Br_Con'])['delq_check'].rolling(window = i).max().values
  transaction = transaction.drop(['delq_check'], axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

# Number of times that a contract has delinquency > 0 day (del >= 1) in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction['delq_check'] = transaction['del'].ge(1).astype(int)
  transaction[f'n_x_dpd_{i}'] = transaction.groupby(['Br_Con'])['delq_check'].rolling(window = i).sum().values
  transaction = transaction.drop(['delq_check'], axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

# Number of times that a contract has delinquency > 30 day (del >= 2) in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction['delq_check'] = transaction['del'].ge(2).astype(int)
  transaction[f'n_30_dpd_{i}'] = transaction.groupby(['Br_Con'])['delq_check'].rolling(window = i).sum().values
  transaction = transaction.drop(['delq_check'], axis = 1)
  print(f'Factor {transaction.columns[-1]} created')
    
# Number of times that a contract has delinquency > 60 day (del >= 3) in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction['delq_check'] = transaction['del'].ge(3).astype(int)
  transaction[f'n_60_dpd_{i}'] = transaction.groupby(['Br_Con'])['delq_check'].rolling(window = i).sum().values
  transaction = transaction.drop(['delq_check'], axis = 1)
  print(f'Factor {transaction.columns[-1]} created')
    
# Number of times that a contract has delinquency > 90 day (del >= 4) in last 12, 9, 6, 3 months
for i in monthRanges:
  transaction['delq_check'] = transaction['del'].ge(4).astype(int)
  transaction[f'n_90_dpd_{i}'] = transaction.groupby(['Br_Con'])['delq_check'].rolling(window = i).sum().values
  transaction = transaction.drop(['delq_check'], axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

# Number of months since last delinquency status as > 0, 30, 60, 90 days
for i, j in zip([1, 2, 3, 4], ['x', '30', '60', '90']):
  transaction['delq_check'] = transaction['del'].ge(i).groupby(transaction['Br_Con']).cumsum()
  transaction[f'n_month_last_{j}'] = transaction.groupby(['Br_Con', 'delq_check']).cumcount().where(transaction['delq_check'].ne(0), 12)
  condition = transaction[f'n_month_last_{j}'] > 12
  transaction[f'n_month_last_{j}'] = np.where(condition, 12, transaction[f'n_month_last_{j}'])
  transaction = transaction.drop(['delq_check'], axis = 1)
  print(f'Factor {transaction.columns[-1]} created')

# Longest run of consecutive months which delinquency status as > 0 days in the last 12, 9, 6, 3 months
for i in monthRanges:
  def consecutiveCount(data, w = i):
    return data.ge(1).rolling(w).apply(maxConsecutive)
  transaction[f'delq_x_run_{i}'] = transaction.groupby('Br_Con')['del'].transform(consecutiveCount)
  print(f'Factor {transaction.columns[-1]} created')
    
# Longest run of consecutive months which delinquency status as > 30 days in the last 12, 9, 6, 3 months
for i in monthRanges:
  def consecutiveCount(data, w = i):
    return data.ge(2).rolling(w).apply(maxConsecutive)
  transaction[f'delq_30_run_{i}'] = transaction.groupby('Br_Con')['del'].transform(consecutiveCount)
  print(f'Factor {transaction.columns[-1]} created')

# Longest run of consecutive months which delinquency status as > 60 days in the last 12, 9, 6, 3 months
for i in monthRanges:
  def consecutiveCount(data, w = i):
    return data.ge(3).rolling(w).apply(maxConsecutive)
  transaction[f'delq_60_run_{i}'] = transaction.groupby('Br_Con')['del'].transform(consecutiveCount)
  print(f'Factor {transaction.columns[-1]} created')

# Longest run of consecutive months which delinquency status as > 90 days in the last 12, 9, 6, 3 months
for i in monthRanges:
  def consecutiveCount(data, w = i):
    return data.ge(4).rolling(w).apply(maxConsecutive)
  transaction[f'delq_90_run_{i}'] = transaction.groupby('Br_Con')['del'].transform(consecutiveCount)
  print(f'Factor {transaction.columns[-1]} created')

Factor max_del_12 created
Factor max_del_9 created
Factor max_del_6 created
Factor max_del_3 created
Factor ever_x_dpd_12 created
Factor ever_x_dpd_9 created
Factor ever_x_dpd_6 created
Factor ever_x_dpd_3 created
Factor ever_30_dpd_12 created
Factor ever_30_dpd_9 created
Factor ever_30_dpd_6 created
Factor ever_30_dpd_3 created
Factor ever_60_dpd_12 created
Factor ever_60_dpd_9 created
Factor ever_60_dpd_6 created
Factor ever_60_dpd_3 created
Factor ever_90_dpd_12 created
Factor ever_90_dpd_9 created
Factor ever_90_dpd_6 created
Factor ever_90_dpd_3 created
Factor n_x_dpd_12 created
Factor n_x_dpd_9 created
Factor n_x_dpd_6 created
Factor n_x_dpd_3 created
Factor n_30_dpd_12 created
Factor n_30_dpd_9 created
Factor n_30_dpd_6 created
Factor n_30_dpd_3 created
Factor n_60_dpd_12 created
Factor n_60_dpd_9 created
Factor n_60_dpd_6 created
Factor n_60_dpd_3 created
Factor n_90_dpd_12 created
Factor n_90_dpd_9 created
Factor n_90_dpd_6 created
Factor n_90_dpd_3 created
Factor n_month_last

In [13]:
# Export factors trasaction table
transaction.to_parquet('factorCreation.parquet', engine = 'pyarrow')